<a href="https://colab.research.google.com/github/ntua-unit-of-control-and-informatics/jaqpot-google-collab-examples/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This example demonstrates a Graph Regression example using Aqueous Solubility dataset.

In [1]:
!pip install PyTDC --quiet
!pip install jaqpotpy --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/4

Import required Libraries

In [6]:
import warnings
import torch
from torch_geometric.loader import DataLoader
from jaqpotpy import Jaqpot
from jaqpotpy.descriptors.graph import SmilesGraphFeaturizer
from jaqpotpy.datasets import SmilesGraphDataset
from jaqpotpy.models.torch_geometric_models.graph_neural_network import GraphSageNetwork, pyg_to_onnx
from jaqpotpy.models.trainers.graph_trainers import RegressionGraphModelTrainer

# This is do disable any rdkit warnings
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
warnings.simplefilter("ignore")

Load the regression data. We use TDC library to obtain train, validation, and test splits with SMILES and Solubility.

In [3]:
from tdc.single_pred import ADME

data = ADME(name = 'Solubility_AqSolDB')
data_splits = data.get_split()

def split_to_list(split):
    return data_splits[split]['Drug'].to_list(), data_splits[split]['Y'].to_list()

# List of smiles and endpoints
train_smiles , train_y =  split_to_list('train')
val_smiles, val_y = split_to_list('valid')
test_smiles , test_y =  split_to_list('test')

Downloading...
100%|██████████| 853k/853k [00:01<00:00, 758kiB/s]
Loading...
Done!


Firstly, a SmilesGraphFeaturizer instance is created. We add 4 features in the featurizer and their specified value (if needed).

In [4]:
featurizer = SmilesGraphFeaturizer(include_edge_features=False)
featurizer.add_atom_feature(
    "symbol",
    ['C', 'K', 'S', 'Br', 'H', 'F', 'N', 'Cl', 'P', 'O', 'I', 'UNK']
)
featurizer.add_atom_feature("total_num_hs", [0, 1, 2, 3, 4])
featurizer.add_atom_feature("degree", [0, 1, 2, 3, 4, 5, 6])
featurizer.add_atom_feature("is_aromatic")

Create train, validation and testing datasets

In [7]:
train_dataset = SmilesGraphDataset(
    smiles=train_smiles, y=train_y, featurizer=featurizer
)
val_dataset = SmilesGraphDataset(
    smiles=val_smiles, y=val_y, featurizer=featurizer
)
test_dataset = SmilesGraphDataset(
    smiles=test_smiles, y=test_y, featurizer=featurizer
)

train_dataset.precompute_featurization()
val_dataset.precompute_featurization()
test_dataset.precompute_featurization()

Create a model. For this demonstration GraphConvolutionNetwork is used with only node features.

In [17]:
node_features = featurizer.get_num_node_features()
model = GraphSageNetwork(
    input_dim=node_features,
    hidden_layers=1,
    hidden_dim=32,
    output_dim=1,
    activation=torch.nn.ReLU(),
    dropout_proba=0.2,
    batch_norm = False,
    seed = 42,
    pooling="add",
)

ReLU()


Define optimizer and loss function

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
loss = torch.nn.MSELoss()

Create instance of Regression Trainer

In [19]:
trainer = RegressionGraphModelTrainer(
    model=model, # Jaqpotpy Graph Model
    n_epochs=20,
    optimizer=optimizer,
    loss_fn=loss,
    scheduler=None
)

Create PyTorch geometric dataloaders

In [20]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

Train and evaluate on the validation set for 20 epochs

In [21]:
trainer.train(train_loader, val_loader)

Epoch 1/20: 100%|██████████| 55/55 [00:01<00:00, 49.21it/s, loss=6.81]


Epoch 1:


INFO:jaqpotpy.models.trainers.base_trainer:Epoch 1:


 Train: loss=6.8124 | explained_variance=0.2953 | r2=0.1846 | mse=4.6464 | rmse=2.1555 | mae=1.5971


INFO:jaqpotpy.models.trainers.base_trainer: Train: loss=6.8124 | explained_variance=0.2953 | r2=0.1846 | mse=4.6464 | rmse=2.1555 | mae=1.5971


...

INFO:jaqpotpy.models.trainers.base_trainer:Epoch 20:


 Train: loss=3.1020 | explained_variance=0.4979 | r2=0.4949 | mse=2.8780 | rmse=1.6965 | mae=1.1720


INFO:jaqpotpy.models.trainers.base_trainer: Train: loss=3.1020 | explained_variance=0.4979 | r2=0.4949 | mse=2.8780 | rmse=1.6965 | mae=1.1720


 Val:   loss=5.1114 | explained_variance=0.0407 | r2=0.0368 | mse=5.1114 | rmse=2.2608 | mae=1.1777


INFO:jaqpotpy.models.trainers.base_trainer: Val:   loss=5.1114 | explained_variance=0.0407 | r2=0.0368 | mse=5.1114 | rmse=2.2608 | mae=1.1777


Evaluate and show metrics on the test dataset

In [22]:
loss, metrics = trainer.evaluate(test_loader)

In [23]:
metrics

{'explained_variance': 0.44174086266339563,
 'r2': 0.43934191910249953,
 'mse': 3.04251677339682,
 'rmse': 1.7442811623694214,
 'mae': 1.1587900009571075,
 'loss': 3.0425167585422614}

Convert PyTorch Geomtric model to onnx format

In [24]:
onnx_model = pyg_to_onnx(model, featurizer)

Log into jaqpot

In [25]:
jaqpot = Jaqpot()
jaqpot.login()

Open this URL in your browser and log in:
https://login.jaqpot.org/realms/jaqpot/protocol/openid-connect/auth?client_id=jaqpot-client&response_type=code&redirect_uri=urn:ietf:wg:oauth:2.0:oob&scope=openid email profile&state=random_state_value
Enter the authorization code you received: 53697dde-08a1-44df-85ca-f255e3b6bb3d.fc75752b-c81e-49ed-b0ff-48cf981ea5e0.40e0db1a-58ce-461a-8fbb-6a4451d8587a


Deploy the model on the web

In [26]:
jaqpot.deploy_torch_model(
    onnx_model,
    featurizer=featurizer,  # Featurizer used for the model
    name="Graph Neural Network",
    description="Graph Sage Network for solubility",
    target_name="SOLUBILITY",
    visibility="PRIVATE",
    task="regression",  # Specify the task (regression or binary_classification)
)

  2024-11-12 08:40:22,427 - INFO - Model has been successfully uploaded. The url of the model is https://app.jaqpot.org/dashboard/models/1911
